In [61]:
import os
import numpy as np
from pathlib import Path

import pandas as pd
import sys
# kouchou-aiのパスを追加
sys.path.append("../kouchou-ai/server")
sys.path.append("../kouchou-ai/server/broadlistening/pipeline/")

In [62]:
slug = "rep-election-2"
exp_prefix = f"20250415_ベースプロンプト_{slug}"
DATA_DIR = Path("../data/original_outputs")  # 広聴AIで出力したデータを保存しているディレクトリ
OUTPUT_DIR = Path("../data/experiment_outputs")

# クラスタリング

In [63]:
from umap import UMAP
from broadlistening.pipeline.steps.hierarchical_clustering import hierarchical_clustering_embeddings

slug = "aipubcom"
arguments_df = pd.read_csv(DATA_DIR / slug /  "args.csv")
embeddings_df = pd.read_pickle(DATA_DIR / slug / "embeddings.pkl")
embeddings_array = np.asarray(embeddings_df["embedding"].values.tolist())
cluster_nums = [15, 70]
n_samples = embeddings_array.shape[0]
default_n_neighbors = 15

if n_samples <= default_n_neighbors:
    n_neighbors = max(2, n_samples - 1)
else:
    n_neighbors = default_n_neighbors

umap_model = UMAP(random_state=42, n_components=2, n_neighbors=n_neighbors)
umap_embeds = umap_model.fit_transform(embeddings_array)


/Users/nasuka/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/nasuka/.pyenv/versions/3.12.4/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [64]:
# 広聴AIで出力したレポートのslug
cluster_results = hierarchical_clustering_embeddings(
    umap_embeds=umap_embeds,
    cluster_nums=cluster_nums,
)
result_df = pd.DataFrame(
    {
        "arg-id": arguments_df["arg-id"],
        "argument": arguments_df["argument"],
        "x": umap_embeds[:, 0],
        "y": umap_embeds[:, 1],
    }
)
for cluster_level, final_labels in enumerate(cluster_results.values(), start=1):
    result_df[f"cluster-level-{cluster_level}-id"] = [f"{cluster_level}_{label}" for label in final_labels]



start initial clustering
end initial clustering
start hierarchical clustering
[15, 70]
n_cluster_cut:  15
end hierarchical clustering


In [65]:
clusters_argument_df =  result_df.copy()
orig_initial_label_df = pd.read_csv(DATA_DIR / slug / "hierarchical_initial_labels.csv")
orig_merge_label_df = pd.read_csv(DATA_DIR / slug / "hierarchical_merge_labels.csv")

exp_output_dir = OUTPUT_DIR / exp_prefix

os.makedirs(exp_output_dir, exist_ok=True)


# プロンプト修正の実験

## 初期ラベリング

In [66]:
INITIAL_PROMPT = """あなたはKJ法が得意なデータ分析者です。userのinputはグループに集まったラベルです。なぜそのラベルが一つのグループであるか解説して、それから表札をつけてください。
出力はJSONとし、フォーマットは以下のサンプルを参考にしてください。


# サンプルの入出力
## 入力例
最近、政治家が能登の復興に向けた具体的なプランを発表し、地域の未来に明るい希望が見えてきました。市民として、真摯な取り組みに感謝しています。
災害復興支援が、選挙期間中にしっかり議論されるようになり、政治家が国民の本当のニーズに応える姿勢に期待しています。
選挙を通じて、政治家が地域振興に全力で取り組む姿勢が伝わってきます。具体的な政策提案を目にするたび、未来への希望が膨らみます。


## 出力例
{{
    "label": "市民の未来を支える具体的政策への期待",
    "description": "このクラスタには、地域の復興や被害者支援など、実際の社会課題に対して政治家が具体的かつ積極的に取り組む姿勢を支持する前向きな意見が集まっています。市民は、選挙や政策議論を通じて、現実の問題に即した支援策や復興計画が実現されることを期待し、明るい未来の構築に向けた政治の変革を応援しています。"
}}"""

In [67]:
from broadlistening.pipeline.steps.hierarchical_initial_labelling import initial_labelling

In [68]:
cluster_id_columns = [col for col in clusters_argument_df.columns if col.startswith("cluster-level-")]
initial_cluster_id_column = cluster_id_columns[-1]
sampling_num = 30
initial_labelling_prompt = INITIAL_PROMPT
model = "gpt-4o"
workers = 70

initial_label_df = initial_labelling(
    initial_labelling_prompt,
    clusters_argument_df,
    sampling_num,
    model,
    workers,
)

In [69]:
# 最下層のクラスタ。プロンプト修正後
initial_label_df.head()

,cluster_id,label,description
0,2_34,生成AIによる著作権侵害とクリエイター保護の必要性,このクラスタには、生成AIがもたらす著作権侵害やクリエイターへの影響に対する懸念が集まってい...
1,2_56,生成AIによる著作権侵害と規制の必要性,このクラスタには、生成AIがもたらす著作権侵害やクリエイターへの影響に対する懸念が集まってい...
2,2_28,生成AIとクリエイターの対立と共存の課題,このクラスタには、生成AIの普及がクリエイターに与える影響についての懸念や批判が集まっていま...
3,2_61,日本文化とクリエイターを守るための生成AI規制の必要性,このクラスタには、生成AIが日本のアニメやマンガなどの文化に与える影響を懸念し、文化を守るた...
4,2_14,生成AIによる創作物の識別と著作権保護への懸念,このクラスタには、生成AIがもたらす創作物の識別困難さと、それに伴う著作権侵害やトラブルへの...


In [70]:
initial_clusters_argument_df = clusters_argument_df.merge(
    initial_label_df,
    left_on=initial_cluster_id_column,
    right_on="cluster_id",
    how="left",
).rename(
    columns={
        "label": f"{initial_cluster_id_column.replace('-id', '')}-label",
        "description": f"{initial_cluster_id_column.replace('-id', '')}-description",
    }
)

In [71]:
initial_clusters_argument_df

,arg-id,argument,x,y,cluster-level-1-id,cluster-level-2-id,cluster_id,cluster-level-2-label,cluster-level-2-description
0,A59d26584-95a8-481b-826e-9a86d30e31d4_0,特定のクリエイターの画風がAIによって複製され、悪意のある創作に利用される可能性があるため、...,0.375076,9.062553,1_8,2_34,2_34,生成AIによる著作権侵害とクリエイター保護の必要性,このクラスタには、生成AIがもたらす著作権侵害やクリエイターへの影響に対する懸念が集まってい...
1,A59d26584-95a8-481b-826e-9a86d30e31d4_1,生成AIによって画風が再現・量産されることで、クリエイターの画風や画力の希少価値が下がり、顧...,1.399587,8.022738,1_12,2_56,2_56,生成AIによる著作権侵害と規制の必要性,このクラスタには、生成AIがもたらす著作権侵害やクリエイターへの影響に対する懸念が集まってい...
2,A59d26584-95a8-481b-826e-9a86d30e31d4_2,生成AIの登場により、活動の意義や需要を失ったクリエイターが活動を断念するケースが増えており...,0.902624,10.476227,1_9,2_28,2_28,生成AIとクリエイターの対立と共存の課題,このクラスタには、生成AIの普及がクリエイターに与える影響についての懸念や批判が集まっていま...
3,A59d26584-95a8-481b-826e-9a86d30e31d4_3,日本のクリエイターの活動と創作活動の発展のために、画風の保護と生成AIの使用に関する規制が必...,1.696006,11.296448,1_7,2_61,2_61,日本文化とクリエイターを守るための生成AI規制の必要性,このクラスタには、生成AIが日本のアニメやマンガなどの文化に与える影響を懸念し、文化を守るた...
4,A4c34e2ee-1e19-47b4-88b1-4d5232fda9f1_0,好きな作家が生成AIに模倣されて創作意欲を失うのは悲しいので、生成AIは規制してほしい,2.454115,4.803548,1_13,2_14,2_14,生成AIによる創作物の識別と著作権保護への懸念,このクラスタには、生成AIがもたらす創作物の識別困難さと、それに伴う著作権侵害やトラブルへの...
...,...,...,...,...,...,...,...,...,...
25225,Ab8495764-38fb-4935-ade0-544c00bc27b6_0,他者の作品を学習素材とした生成AIの使用は違法とすべき,-1.027124,8.351385,1_8,2_55,2_55,生成AIによる著作権侵害とクリエイターの権利保護の必要性,このクラスタには、生成AIが特定の作家の作品を無断で学習し、著作権を侵害することに対する懸念...
25226,Ab8495764-38fb-4935-ade0-544c00bc27b6_1,生成AIは他者の著作物のモンタージュに過ぎず、盗作の寄せ集めである,-0.913003,10.382530,1_10,2_59,2_59,生成AIと著作権に関する倫理的・法的課題,このクラスタには、生成AIの利用における著作権侵害の懸念や倫理的問題に関する意見が集まってい...
25227,Ab8495764-38fb-4935-ade0-544c00bc27b6_2,生成AIの使用を許可すると創作業界が模造コピー品で溢れる,-0.080795,9.903406,1_9,2_2,2_2,生成AIによる著作権侵害と文化への影響に対する懸念,このクラスタには、生成AIが著作権を侵害し、クリエイターの権利や創作意欲を損なうことに対する...
25228,Ab8495764-38fb-4935-ade0-544c00bc27b6_3,生成AIを用いる場合、学習素材は製作者自身の創作物に限定すべき,-1.398726,8.439795,1_8,2_8,2_8,生成AIと著作権に関する倫理的・法的懸念,このクラスタには、生成AIが著作物を無断で学習することに対する倫理的および法的な懸念が集まっ...


In [72]:
initial_clusters_argument_df.to_csv(exp_output_dir / "hierarchical_initial_labels.csv", index=False)

## 統合ラベリング

In [73]:
from broadlistening.pipeline.steps.hierarchical_merge_labelling import _filter_id_columns, merge_labelling, melt_cluster_data, _build_parent_child_mapping, calculate_cluster_density

In [74]:
MERGE_PROMPT = """分割されすぎたクラスタを統合する必要があるので、統合後の名称を考えて出力して。

# 指示
* 統合前のクラスタの名称・説明および統合後のクラスタに属するデータ点のサンプルを与えるので、これらに基づいて統合後のクラスタの名称を出力してください
    * 統合後のクラスタ名において、統合前のクラスタ名をそのまま使うことは避けてください。
* 出力例に記載したJSONのフォーマットに従って出力してください

# サンプルの入出力
## 入力例（クラスタラベル:説明文）
- 地域の災害対応への批判: このクラスタは、地域における災害対応策の実施や体制に対する批判的な意見を集約したものです。住民からは、迅速かつ効果的な支援が行われていない点や、情報提供・連携の不足などに対する強い不満が表明されています。
- 災害対応への不満: このクラスタは、災害発生時の対応全般に対する不満を示す意見をまとめたものです。救援活動の遅れや支援策の実効性に疑問を持つ声が多く、より積極的で透明性のある対応を求める意見が特徴です。
- 地域復興の遅れ: このクラスタは、災害後の地域復興プロセスが予定通りに進んでいない点に対する懸念や不満を反映しています。再建計画や支援策の実施の遅延、そしてそれに伴う住民の生活再建への影響が強調されています。


## 出力例
{{
    "label": "地域再生と災害支援に対する期待と懸念",
    "description": "このクラスタは、特定の地域における再生や災害支援策に対し、具体的な取り組みが不足しているとの意見を集約しています。市民は、選挙を通じた政策議論の中で、地域復興や被災者支援を最優先すべきだとの期待と、現行の支援策に対する改善要求を強く表明しており、より効果的な政府の対応を求める声が反映されています。"
}}"""

In [75]:
config = {}
config["hierarchical_merge_labelling"] = {
    "prompt": MERGE_PROMPT,
    "model": "gpt-4o",
    "sampling_num": 50,
    "workers": 70,
}

In [76]:
clusters_df = initial_clusters_argument_df.copy()
cluster_id_columns: list[str] = _filter_id_columns(clusters_df.columns)
merge_result_df = merge_labelling(
    clusters_df=clusters_df,
    cluster_id_columns=sorted(cluster_id_columns, reverse=True),
    config=config,
)

100%|██████████| 1/1 [00:11<00:00, 11.95s/it]


In [77]:
melted_df = melt_cluster_data(merge_result_df)
parent_child_df = _build_parent_child_mapping(merge_result_df, cluster_id_columns)
melted_df = melted_df.merge(parent_child_df, on=["level", "id"], how="left")
melted_df.head(20)

,level,id,label,description,value,parent
0,1,1_8,生成AIと著作権保護に関する倫理的・法的課題,このクラスタは、生成AIの利用に伴う著作権侵害の懸念と、クリエイターの権利保護の必要性に関す...,2557,0
1,1,1_12,生成AIとクリエイターの権利保護に関する倫理的および法的課題,このクラスタは、生成AI技術の進化がもたらすクリエイターへの影響や著作権侵害の懸念を集約して...,2323,0
2,1,1_9,生成AIとクリエイティブ産業の倫理的調和と権利保護,このクラスタは、生成AIの普及がクリエイティブ産業に与える影響についての懸念と、倫理的な調和...,1837,0
3,1,1_7,生成AIの規制と文化保護に関する懸念と提案,このクラスタは、生成AI技術の急速な発展が日本の文化やクリエイターに与える影響に対する懸念を...,2848,0
4,1,1_13,生成AIによる創作物の識別と著作権保護への懸念,このクラスタには、生成AIがもたらす創作物の識別困難さと、それに伴う著作権侵害やトラブルへの...,231,0
5,1,1_15,AI技術とクリエイターの権利保護に関する懸念と対策,このクラスタは、AI技術の進化がクリエイターの権利や創作活動に与える影響についての懸念を集約...,1842,0
6,1,1_4,AI技術における著作権とデータ利用の倫理的・法的課題,このクラスタは、AI技術の発展に伴う著作権とデータ利用に関する倫理的および法的な課題を集約し...,2258,0
7,1,1_2,日本の創作文化とクリエイターの権利保護への期待と課題,このクラスタは、日本の創作文化を支えるクリエイターの権利保護の重要性と、それに伴う法的整備の...,1618,0
8,1,1_14,生成AIの倫理と法規制に関する懸念と提案,このクラスタは、生成AI技術の進化に伴う倫理的および法的な問題に対する懸念を集約しています。...,1951,0
9,1,1_1,創作活動と著作権保護に関する法的課題と倫理的懸念,このクラスタは、創作活動における著作権保護の重要性と、それに伴う法的および倫理的な課題に対す...,1826,0


In [78]:
melted_df.to_csv(exp_output_dir / "プロンプト変更後のラベリング結果.csv", index=False)

In [79]:
# 自動評価

In [80]:
import json
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
random.seed(42)

from broadlistening.pipeline.services.llm import request_to_chat_openai

In [81]:
EVALUATION_PROMPT = """あなたはテキストクラスタリングの専門家です。クラスタリング結果のタイトルの適切性を評価してください。

# 指示
与えられた情報に基づいて、クラスタのタイトルがそのクラスタに含まれるテキストデータをどれだけ適切に表現しているかを評価してください。
評価結果は出力例に記載したjson形式で出力してください。

# 入力情報
1. 評価対象クラスタのタイトル
2. 評価対象クラスタに所属するテキストデータのサンプル
3. 他のクラスタからサンプリングしたテキストデータ

# 評価基準
以下の観点から、クラスタのタイトルの適切性を1〜5の5段階で評価してください：

1. inclusiveness_score: タイトルがクラスタ内のすべてのテキストの共通テーマを捉えているか（1: 全く捉えていない 〜 5: 完全に捉えている）
2. specificity_score: タイトルがこのクラスタを他のクラスタと明確に区別できるか（1: 全く区別できない 〜 5: 完全に区別できる）
3. concreteness_score: タイトルが抽象的すぎず、具体的な内容を示しているか（1: 非常に抽象的 〜 5: 非常に具体的）

# 評価例

## 入力例
評価対象クラスタのタイトル: 「災害対応の迅速化と情報共有の強化」

評価対象クラスタに所属するテキストデータのサンプル:
- 災害発生時の初動対応をもっと迅速にしてほしい。特に高齢者への支援が遅れがちだ。
- 避難所の情報がリアルタイムで更新されず、どこに行けばいいのか混乱した。
- 災害時の情報共有システムを強化し、住民が必要な情報にすぐアクセスできるようにすべき。
- 地域ごとの災害対策本部の連携が不十分で、支援物資の配布に無駄が生じていた。
- 災害発生から避難指示が出るまでのタイムラグを短縮する必要がある。

他のクラスタからサンプリングしたテキストデータ:
- 復興予算の使い道をもっと透明化してほしい。どこにいくら使われているのか不明確だ。
- 被災地域の経済復興のための長期的な計画が見えてこない。
- 防災教育を学校カリキュラムに積極的に取り入れるべきだ。
- ボランティアの受け入れ体制が整っておらず、せっかくの支援の手が活かされていない。

## 出力例
{{
  "inclusiveness_score": {{
      "score": 4,
      "reason": "タイトル「災害対応の迅速化と情報共有の強化」は、クラスタ内のテキストの主要なテーマである「初動対応の迅速化」「情報共有の問題」「対策本部の連携」などの要素を広くカバーしています。ただし、「高齢者への支援」という具体的な対象については明示されていないため、完全ではありません。"
  }},
  "specificity_score": {{
      "score": 4,
      "reason": "このタイトルは「対応の迅速化」と「情報共有」という2つの明確な焦点を持っており、他のクラスタ（予算の透明化、経済復興、防災教育など）とは明確に区別できます。ただし、災害対応の中でも「初動」に関する内容が多いため、もう少し特化した表現があるとより区別しやすくなります。"
  }},
  "concreteness_score": {{
      "score": 3,
      "reason": "「迅速化」と「情報共有の強化」という方向性は示されていますが、どのような迅速化（初動対応、避難指示など）なのか、どのような情報共有（避難所情報、支援物資など）なのかまでは具体化されていません。"
  }},
}}
"""

In [82]:
def sample_cluster_data(
    df: pd.DataFrame,
    cluster_column: str,
    cluster_id: str,
    text_column: str,
    sample_size: int
) -> list[str]:
    """クラスタからデータをサンプリングする

    Args:
        df: クラスタリング結果のDataFrame
        cluster_column: クラスタIDが格納されている列名
        cluster_id: サンプリング対象のクラスタID
        text_column: テキストデータが格納されている列名
        sample_size: サンプリングするデータ数

    Returns:
        サンプリングしたテキストデータのリスト
    """
    cluster_data = df[df[cluster_column] == cluster_id]
    sample_size = min(sample_size, len(cluster_data))
    
    if sample_size == 0:
        return []
    
    return cluster_data.sample(sample_size)[text_column].tolist()


def sample_other_clusters_data(
    df: pd.DataFrame,
    cluster_column: str,
    exclude_cluster_id: str,
    text_column: str,
    sample_size: int,
    sample_clusters: int | None = None
) -> list[str]:
    """他のクラスタからデータをサンプリングする

    Args:
        df: クラスタリング結果のDataFrame
        cluster_column: クラスタIDが格納されている列名
        exclude_cluster_id: 除外するクラスタID
        text_column: テキストデータが格納されている列名
        sample_size: 各クラスタからサンプリングするデータ数
        sample_clusters: サンプリングするクラスタ数（Noneの場合は全クラスタ）

    Returns:
        サンプリングしたテキストデータのリスト
    """
    other_clusters = df[df[cluster_column] != exclude_cluster_id][cluster_column].unique()
    
    if len(other_clusters) == 0:
        return []
    
    if sample_clusters is not None and sample_clusters < len(other_clusters):
        other_clusters = random.sample(list(other_clusters), sample_clusters)
    
    sampled_data = []
    for cluster_id in other_clusters:
        cluster_samples = sample_cluster_data(
            df, cluster_column, cluster_id, text_column, sample_size
        )
        sampled_data.extend(cluster_samples)
    
    return sampled_data


def evaluate_cluster_title(
    prompt: str,
    cluster_title: str,
    cluster_samples: list[str],
    other_clusters_samples: list[str],
    model: str = "gpt-4o"
) -> dict[str, dict]:
    """クラスタのタイトルを評価する

    Args:
        prompt: 評価用プロンプト
        cluster_title: 評価対象のクラスタタイトル
        cluster_samples: 評価対象クラスタのサンプルデータ
        other_clusters_samples: 他のクラスタのサンプルデータ
        model: 使用するLLMモデル名

    Returns:
        評価結果（JSON形式）
    """
    # 入力データの整形
    input_text = f"評価対象クラスタのタイトル: 「{cluster_title}」\n\n"
    
    input_text += "評価対象クラスタに所属するテキストデータのサンプル:\n"
    for sample in cluster_samples:
        input_text += f"- {sample}\n"
    
    input_text += "\n他のクラスタからサンプリングしたテキストデータ:\n"
    for sample in other_clusters_samples:
        input_text += f"- {sample}\n"
    
    # LLMに評価リクエスト
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": input_text},
    ]
    
    try:
        response = request_to_chat_openai(messages=messages, model=model, is_json=True)
        evaluation = json.loads(response)
        return evaluation
    except Exception as e:
        print(f"評価中にエラーが発生しました: {e}")
        return {
            "error": str(e),
            "title_evaluation": {
                "inclusiveness_score": {"score": 0, "reason": "評価エラー"},
                "specificity_score": {"score": 0, "reason": "評価エラー"},
                "concreteness_score": {"score": 0, "reason": "評価エラー"},
                "overall_score": {"score": 0, "reason": "評価エラー"},
            }
        }

def evaluate(
    prompt: str,
    clusters_df: pd.DataFrame,
    cluster_column: str,
    text_column: str,
    sample_size: int,
    other_sample_size: int,
    other_sample_clusters: int | None = None,
    max_workers: int | None = None,  # 並列数を指定（Noneなら自動）
) -> list[dict[str, dict]]:
    cluster_ids = sorted(clusters_df[cluster_column].unique())
    evaluation_results = []

    def eval_one(cluster_id):
        cluster_title = clusters_df[clusters_df[cluster_column] == cluster_id]["cluster-level-2-label"].values[0]
        cluster_samples = sample_cluster_data(clusters_df, cluster_column, cluster_id, text_column, sample_size)
        other_clusters_samples = sample_other_clusters_data(
            clusters_df, cluster_column, cluster_id, text_column, other_sample_size, other_sample_clusters
        )
        return {
            "id": cluster_id,
            "label": cluster_title,
            **evaluate_cluster_title(prompt, cluster_title, cluster_samples, other_clusters_samples)
        }

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(eval_one, cluster_id): cluster_id for cluster_id in cluster_ids}
        for future in tqdm(as_completed(futures), total=len(futures)):
            result = future.result()
            evaluation_results.append(result)

    return evaluation_results


In [83]:
cluster_column = "cluster-level-1-id"
cluster_ids = sorted(clusters_df[cluster_column].unique())


In [84]:
eval_results = evaluate(
    prompt=EVALUATION_PROMPT,
    clusters_df=clusters_df,
    cluster_column="cluster-level-1-id",
    text_column="argument",
    sample_size=30,
    other_sample_size=4,
    other_sample_clusters=10
)

100%|██████████| 15/15 [00:11<00:00,  1.35it/s]


In [85]:
def parse_val(val: dict | str):
    if isinstance(val, dict):
        return val["score"]
    return val

In [86]:
eval_score_df = pd.DataFrame([
        {
            key: parse_val(val_dict)
            for key, val_dict in eval_result.items()} 
            for eval_result in eval_results
            
    ]
)

In [87]:
eval_score_df.sort_values(by="id")

,id,label,inclusiveness_score,specificity_score,concreteness_score
11,1_1,著作権保護の強化と法整備の必要性,5,4,4
4,1_10,生成AIと著作権に関する倫理的・法的懸念,5,4,4
5,1_11,生成AIと著作権侵害に関する懸念,5,4,4
10,1_12,生成AIによる著作権侵害と規制の必要性,5,4,4
9,1_13,生成AIによる創作物の識別と著作権保護への懸念,5,4,4
12,1_14,生成AIの透明性と規制の必要性,5,4,4
13,1_15,無断学習と著作権侵害に対する具体的対策の必要性,5,4,4
14,1_2,クリエイターの権利保護と文化の持続的発展の必要性,5,4,4
6,1_3,AIと著作権に関する懸念と規制の必要性,5,4,4
0,1_4,著作権と生成物に関する懸念と規制の必要性,5,4,4


In [88]:
eval_score_df[["inclusiveness_score", "specificity_score", "concreteness_score"]].mean(axis=0)

inclusiveness_score    4.933333
specificity_score      4.000000
concreteness_score     3.800000
dtype: float64

In [89]:
print("平均スコア", eval_score_df[["inclusiveness_score", "specificity_score", "concreteness_score"]].mean().mean())

平均スコア 4.2444444444444445


In [90]:
eval_score_df.sort_values(by="id").to_csv(exp_output_dir / "ベースプロンプトの自動評価結果.csv", index=False)